In [10]:
#importar bibliotecas
import dash
from dash import dcc
from dash import html

import plotly.graph_objs as go

import pandas as pd
import math

from dash.dependencies import Input, Output

In [11]:
#!/usr/bin/python

#FUNÇÃO LINEAR

# define layout
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div(children=[  

    # cabeçalho
    html.H1(children = 'Linear function', style={'color':'red'}),    

    # plot 
    dcc.Graph(
        figure = {
                'data': [go.Scatter(x = pd.Series(range(-100, 100, 1)), 
                                    y = pd.Series(range(-100, 100, 1)), 
                                    mode = 'lines',
                                    name = 'linear_func')],
                'layout': go.Layout(xaxis = {'title': 'x'},
                                    yaxis = {'title': 'y'})
             },      
        id = 'linear_func_id'
    ),         
  
])
# lógica do dashboard
if __name__ == '__main__':
    app.run_server(debug=True)

In [20]:
#SENO E COSSENO COM BOTAO

# defina o layout
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div(
    children=[

        # usando html para criar o cabeçalho
        html.H1(children='Trigonometric functions', style={'color':'red'}),

        #usando html para criar o botão
        html.Label('Display Mode', style={'color':'red'}),
        dcc.RadioItems(
            options=[
                {'label':'Display sin(x)', 'value':'sin'},
                {'label':'Display cos(x)', 'value':'cos'},
            ],
            value = 'sin',
            id = 'mode_selector',
            inline=True,
        ),

        #o botao chama o gráfico
        dcc.Graph(
            id = 'trig_func'
        ),

    ]
)

#lógica do dashboard
@app.callback(
    [
        Output('trig_func', 'figure'),
    ],
    [
        Input('mode_selector', 'value'),
    ],
)


def update_figures(selected_mode):
    
    #gerando os valores x e y para os dados da função
    x = range(-100, 100, 1)
    x = [x / 10 for x in x]
    y = [math.sin(x) for x in x]
    if selected_mode == 'cos':
        y = [math.cos(x) for x in x]
    

    #gerar o gráfico que sera parte do retorno da funcao
    data = [
        go.Scatter(
            x=pd.Series(x), 
            y=pd.Series(y), 
            mode='lines', 
            name=selected_mode + '(x)',
        ),
    ]   


    #retorno da função
    return(
        {
            'data' : data,
            'layout' : go.Layout(xaxis={'title': 'x'}, yaxis={'title': 'y'})
        },
    )



# lógica do dashboard
if __name__ == '__main__':
    app.run_server(debug=True)

In [13]:
# defina os dados a serem exibidos
games_raw = pd.read_csv('datasets/vgsales.csv')

games_raw = games_raw.rename(columns={'Year':'Year_of_Release'})

games_raw = games_raw.dropna(subset=['Year_of_Release'])
games_raw['Year_of_Release'] = pd.to_datetime(games_raw['Year_of_Release'].astype(int).astype(str), format='%Y')

games_raw

,Rank,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006-01-01,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985-01-01,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008-01-01,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009-01-01,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996-01-01,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002-01-01,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003-01-01,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008-01-01,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010-01-01,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


In [14]:




# defina os dados para o relatório
games_grouped = (
    games_raw.groupby(['Genre', 'Year_of_Release'])
    .agg({'Name': 'count'})
    .reset_index()
    .rename(columns={'Name': 'Games Released'})
)

# defina os gráficos a serem exibidos
data = []
for genre in games_grouped['Genre'].unique():
    current = games_grouped.query('Genre == @genre')
    data += [
        go.Scatter(
            x=current['Year_of_Release'],
            y=current['Games Released'],
            mode='lines',
            stackgroup='one',
            name=genre,
        )
    ]
# defina o layout
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(
    __name__, external_stylesheets=external_stylesheets, compress=False
)
app.layout = html.Div(
    children=[
        # html
        html.H1(children='Games released by year'),
        dcc.Graph(
            figure={
                'data': data,
                'layout': go.Layout(
                    xaxis={'title': 'Year'}, yaxis={'title': 'Games released'}
                ),
            },
            id='games_by_year',
        ),
    ]
)

# lógica do dashboard
if __name__ == '__main__':
    app.run_server(debug=True)